Note:

Forget “perplexity”.
For domain SLMs, you want to track:

Metric	Description
Exact match rate	% of tokens matching valid domain grammar
Parse accuracy	% of generated outputs that compile/execute
Semantic validity	% of outputs that make logical sense
Entropy per token	How confident vs overfitted model is

In [28]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("YvvonM/mental_health_data")
train_dataset = dataset['train'].train_test_split(test_size=0.1)['train']
eval_dataset = dataset['train'].train_test_split(test_size=0.1)['test']

In [ ]:
train_batch_size = 8  # training batch size
eval_batch_size = 4  # evaluation batch size
context_length = block_size = 126  # number of tokens processed in a single batch
train_split = 0.9  # percentage of data to use from total data for training
number_of_heads = 8
number_of_layers = 6

In [31]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast(
    tokenizer_file="../slm/tokenizer.json",
    truncation=True,
    unk_token="<UNK>",
    pad_token="<PAD>",
    bos_token="<BOS>",
    eos_token="<EOS>",
    max_length=context_length
)

In [32]:

def tokenize_and_shift_labels(example):
    text = f"<BOS> <USER> {example['Input']} <ASSISTANT> {example['Response']} <EOS>"
    encoding = tokenizer(
        text,
        truncation=True,
        max_length=context_length,
        padding='max_length'
    )
    input_ids = encoding["input_ids"]

    inputs = input_ids[:-1]
    labels = input_ids[1:]
    labels = [l if l != tokenizer.pad_token_id else -100 for l in labels]

    return {"input_ids": inputs, "labels": labels}

In [33]:
from random import shuffle
train_dataset = train_dataset.map(
    tokenize_and_shift_labels,
    remove_columns=train_dataset.column_names,
)

eval_dataset = eval_dataset.map(
    tokenize_and_shift_labels,
    remove_columns=eval_dataset.column_names,
)

Map:   1%|          | 12078/1586505 [00:12<26:38, 984.86 examples/s] 


KeyboardInterrupt: 

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

device = "cpu"

if torch.cuda.is_available():
  device = "cuda"

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # causal LM
)

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=data_collator)
eval_loader = DataLoader(eval_dataset, batch_size=8, shuffle=False, collate_fn=data_collator)


In [34]:
import math

class PositionalEncoding(nn.Module):
    def __init__(self, context_length, d_model):
        super().__init__()
        self.register_buffer("pe", self._build_pe(context_length, d_model))
        self.dropout = nn.Dropout(dropout=0.1)



    def _build_pe(self, length, d_model):
        position = torch.arange(0, length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe = torch.zeros(length, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        return pe.unsqueeze(0)  # shape: [1, length, d_model]

    def forward(self, x):
        seq_len = x.size(1)
        return self.dropout(x + self.pe[:, :seq_len, :])


In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()
        assert d_model % n_heads == 0
        self.n_heads = n_heads
        self.head_dim = d_model // n_heads

        self.qkv_proj = nn.Linear(d_model, 3 * d_model)
        self.fc_out = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        B, T, C = x.shape
        qkv = self.qkv_proj(x)
        q, k, v = qkv.chunk(3, dim=-1)

        # Reshape for heads
        q = q.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        k = k.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        v = v.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)

        # Scaled dot-product attention
        attn_scores = (q @ k.transpose(-2, -1)) / math.sqrt(self.head_dim)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, float("-inf"))
        attn_weights = F.softmax(attn_scores, dim=-1)
        attn_output = attn_weights @ v

        attn_output = attn_output.transpose(1, 2).contiguous().view(B, T, C)
        return self.fc_out(self.dropout(attn_output))

In [ ]:
class GPTBlock(nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.ln2 = nn.LayerNorm(d_model)
        self.attn = MultiHeadAttention(d_model, n_heads, dropout)
        self.mlp = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.GELU(),
            nn.Linear(4 * d_model, d_model),
            nn.Dropout(dropout)
        )

    def forward(self, x, mask):
        x = x + self.attn(self.ln1(x), mask)
        x = x + self.mlp(self.ln2(x))
        return x


In [ ]:
class GPT(nn.Module):
    def __init__(self, vocab_size, d_model, n_layers, n_heads, context_length, dropout=0.1):
        super().__init__()
        self.context_length = context_length
        self.wte = nn.Embedding(vocab_size, d_model)
        self.wpe = PositionalEncoding(context_length, d_model)

        self.blocks = nn.ModuleList([
            GPTBlock(d_model, n_heads, dropout) for _ in range(n_layers)
        ])
        self.ln_f = nn.LayerNorm(d_model)
        self.lm_head = nn.Linear(d_model, vocab_size, bias=False)
        self.lm_head.weight = self.wte.weight  # Weight tying

        nn.init.normal_(self.wte.weight, mean=0.0, std=0.02)


    def forward(self, x, targets=None):
        B, T = x.size()
        assert T <= self.context_length, f"Input length {T} exceeds context length {self.context_length}"

        mask = torch.tril(torch.ones(T, T, device=x.device)).unsqueeze(0).unsqueeze(0)
        x = self.wte(x)
        x = self.wpe(x)

        for block in self.blocks:
            x = block(x, mask)

        x = self.ln_f(x)
        logits = self.lm_head(x)  # [B, T, vocab_size]

        loss = None
        if targets is not None:
            # Trim targets/logits to same length
            min_len = min(logits.size(1), targets.size(1))
            logits = logits[:, :min_len, :]
            targets = targets[:, :min_len]

            # Flatten for cross-entropy
            loss = F.cross_entropy(
                logits.reshape(-1, logits.size(-1)),
                targets.reshape(-1),
                ignore_index=-100,
                label_smoothing=0.1
            )

        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, top_k=50, top_p=0.9, temperature=1.0, repetition_penalty=1.1):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.context_length:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / temperature

            # Repetition penalty
            for i in range(idx.size(0)):
                for token in idx[i].tolist():
                    logits[i, token] /= repetition_penalty

            # Top-k
            if top_k is not None:
                values, _ = torch.topk(logits, top_k)
                min_values = values[:, -1].unsqueeze(1)
                logits = torch.where(logits < min_values, float("-inf"), logits)

            # Top-p
            if top_p is not None:
                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                probs = F.softmax(sorted_logits, dim=-1)
                cumulative_probs = torch.cumsum(probs, dim=-1)
                sorted_mask = cumulative_probs > top_p
                sorted_mask[:, 1:] = sorted_mask[:, :-1].clone()
                sorted_mask[:, 0] = 0
                logits.scatter_(1, sorted_indices, logits.masked_fill(sorted_mask, float("-inf")))

            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, next_token), dim=1)
        return idx


In [ ]:
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 10
train_batch_size = 16
eval_batch_size = 8
eval_steps = 2000
log_interval = 10  # log every n steps
save_dir = "./checkpoints"
os.makedirs(save_dir, exist_ok=True)


In [ ]:
basic_model = GPT(vocab_size=8000, d_model=256, n_heads=number_of_heads, n_layers=number_of_layers, context_length=context_length).to(device)
# basic_model = GPT(vocab_size=8000, d_model=256).to(device)




basic_model.to(device)
optimizer = torch.optim.AdamW(basic_model.parameters(), lr=3e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs*len(train_loader))


In [ ]:
print(basic_model)
print(f"Total Parameters: {round(sum(p.numel() for p in basic_model.parameters() if p.requires_grad) / 1_000_000)}M")

GPT(
  (wte): Embedding(8000, 256)
  (wpe): PositionalEncoding()
  (blocks): ModuleList(
    (0-5): 6 x GPTBlock(
      (ln1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (attn): MultiHeadAttention(
        (qkv_proj): Linear(in_features=256, out_features=768, bias=True)
        (fc_out): Linear(in_features=256, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (mlp): Sequential(
        (0): Linear(in_features=256, out_features=1024, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=1024, out_features=256, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (lm_head): Linear(in_features=256, out_features=8000, bias=False)
)
Total Parameters: 7M


In [ ]:
xb = next(iter(train_loader))
xb_ids = xb['input_ids'].to(device)
x_embed = basic_model.wte(xb_ids)
print("Embedding stats:", x_embed.min().item(), x_embed.max().item(), x_embed.mean().item())

Embedding stats: -0.08833513408899307 0.08442691713571548 0.00016734316886868328


In [ ]:
x_pos = basic_model.wpe(x_embed)
print("After positional encoding:", x_pos.min().item(), x_pos.max().item(), x_pos.mean().item())


After positional encoding: -1.058415174484253 1.0844268798828125 0.39544805884361267


In [ ]:
mask = torch.tril(torch.ones(xb_ids.size(1), xb_ids.size(1), device=device)).unsqueeze(0).unsqueeze(0)
x = x_embed + x_pos

for i, block in enumerate(basic_model.blocks):
    x = block(x, mask)
    print(f"After block {i}: min={x.min().item()}, max={x.max().item()}, mean={x.mean().item()}")


After block 0: min=-2.10876202583313, max=2.4862170219421387, mean=0.41785746812820435
After block 1: min=-2.5692265033721924, max=3.157790422439575, mean=0.3904191255569458
After block 2: min=-2.949037551879883, max=3.3138914108276367, mean=0.4024883806705475
After block 3: min=-3.5521295070648193, max=3.7736759185791016, mean=0.3811834752559662
After block 4: min=-3.6067259311676025, max=3.7547340393066406, mean=0.38399630784988403
After block 5: min=-3.926696300506592, max=3.9288580417633057, mean=0.3636806607246399


In [ ]:
x_ln = basic_model.ln_f(x)
logits = basic_model.lm_head(x_ln)
print("Final logits stats:", logits.min().item(), logits.max().item(), logits.mean().item())


Final logits stats: -1.5039607286453247 1.6910687685012817 0.003899493021890521


In [ ]:

logits, loss = basic_model(xb['input_ids'].to(device), xb['labels'].to(device))
print(logits.shape, loss.item())

torch.Size([16, 49, 8000]) 8.90952205657959


In [ ]:
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm

scaler = GradScaler()
accum_steps = 4
eval_subset = 100
log_interval = 10000

train_losses, eval_losses = [], []

for epoch in range(epochs):
    basic_model.train()
    running_loss = 0.0
    step_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=True)

    for step, batch in enumerate(step_bar):
        xb = batch['input_ids'].to(device, non_blocking=True)
        yb = batch['labels'].to(device, non_blocking=True)

        with autocast():
            logits, loss = basic_model(xb, yb)
            loss = loss / accum_steps

        scaler.scale(loss).backward()
        running_loss += loss.item()

        if (step + 1) % accum_steps == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(basic_model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()

        # Show loss in progress bar occasionally
        if (step + 1) % log_interval == 0:
            avg_loss = running_loss / log_interval
            step_bar.set_postfix(train_loss=f"{avg_loss:.4f}")
            train_losses.append(avg_loss)
            running_loss = 0.0

        # Evaluate periodically
        if (step + 1) % eval_steps == 0:
            basic_model.eval()
            eval_loss = 0.0
            with torch.no_grad():
                for eval_step, eval_batch in enumerate(eval_loader):
                    if eval_step >= eval_subset:
                        break
                    xvb = eval_batch['input_ids'].to(device, non_blocking=True)
                    yvb = eval_batch['labels'].to(device, non_blocking=True)
                    with autocast():
                        _, e_loss = basic_model(xvb, yvb)
                    eval_loss += e_loss.item()
            eval_loss /= eval_subset
            tqdm.write(f"[Eval] Epoch {epoch+1}, Step {step+1}: Loss = {eval_loss:.4f}")
            eval_losses.append(eval_loss)
            basic_model.train()

    # Save checkpoint at the end of epoch
    checkpoint_path = os.path.join(save_dir, f"slm_epoch_{epoch+1}.pt")
    torch.save({
        'epoch': epoch+1,
        'model_state_dict': basic_model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'train_losses': train_losses,
        'eval_losses': eval_losses
    }, checkpoint_path)
    tqdm.write(f"Checkpoint saved at {checkpoint_path}")


C:\Users\dipgh\AppData\Local\Temp\ipykernel_18492\9814107.py:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch 1/10:   0%|          | 0/99157 [00:00<?, ?it/s]C:\Users\dipgh\AppData\Local\Temp\ipykernel_18492\9814107.py:20: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1/10:   2%|▏         | 1996/99157 [01:08<1:00:39, 26.70it/s, train_loss=0.0026]C:\Users\dipgh\AppData\Local\Temp\ipykernel_18492\9814107.py:52: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1/10:   2%|▏         | 2003/99157 [01:09<2:57:00,  9.15it/s, train_loss=0.0026]

[Eval] Epoch 1, Step 2000: Loss = 0.0050


Epoch 1/10:   4%|▍         | 4003/99157 [02:19<2:53:33,  9.14it/s, train_loss=0.0013]

[Eval] Epoch 1, Step 4000: Loss = 0.0023


Epoch 1/10:   6%|▌         | 6002/99157 [03:30<3:51:30,  6.71it/s, train_loss=0.0007]

...

[Eval] Epoch 6, Step 24000: Loss = 0.0000


Epoch 6/10:  26%|██▌       | 26003/99157 [6:12:27<2:02:59,  9.91it/s, train_loss=0.0000]      

[Eval] Epoch 6, Step 26000: Loss = 0.0000


Epoch 6/10:  28%|██▊       | 27595/99157 [6:13:22<16:08:17,  1.23it/s, train_loss=0.0000]


KeyboardInterrupt: 

In [ ]:
total_tokens = 0
for batch in train_loader:
    total_tokens += batch['input_ids'].numel()
    print(f"Processed {total_tokens:,} tokens", end='\r')
print(f"Total tokens in dataset: {total_tokens:,}")


Total tokens in dataset: 77,738,745
